<a href="https://colab.research.google.com/github/rashida048/ApacheSpark-BigDataAnalytics/blob/main/KNN_Classifier_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
import re
import numpy as np

from numpy import dot
from numpy.linalg import norm

In [2]:
!pip install pyspark

     |████████████████████████████████| 212.4 MB 70 kB/s 
     |████████████████████████████████| 198 kB 67.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=264834064aff64877c55742476eca96dda4959954cb73aadc128ba6f60ed86ea
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [3]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate() 

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [4]:
wikiCategory = sc.textFile("wiki-categorylinks-small.csv.bz2")
wikiPages = sc.textFile("WikipediaPagesOneDocPerLine1000LinesSmall.txt.bz2")

In [5]:
wikiCats = wikiCategory.map(lambda x: x.split(",")).map(lambda x: (x[0].replace('"', ''), x[1].replace('"', '')))
validLines = wikiPages.filter(lambda x: 'id' in x and 'url=' in x)

In [6]:
numberOfDocs = wikiPages.count()
numberOfDocs

1000

In [7]:
keyAndText = validLines.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('">') + 2:][:-6])) 

In [8]:
keyAndText.take(2)

[('431949',
  'Black people and MormonismWhile at least two black men held the priesthood in the early church, from the mid-1800s until 1978, The Church of Jesus Christ of Latter-day Saints (LDS Church) had a policy which prevented most men of black African descent from being ordained to the church\'s lay priesthood. Under the temple and priesthood restrictions before 1978, black members of African descent could not receive the priesthood or participate in temple ordinances besides baptisms for the dead. For a time in the 1960s and 1970s, they were not allowed to perform baptisms for the dead either. For young men and men in the LDS church, holding the priesthood is required to hold leadership roles, perform baptisms, to bless the sacrament, to bless babies and to bless the sick. Since black men could not hold the priesthood, they were excluded from holding leadership roles and performing these rituals. Temple ordinances are necessary for members to receive the endowment and marriage s

In [9]:
regex = re.compile('[^a-zA-Z]')
keyAndListOfWords = keyAndText.map(lambda x : (str(x[0]), regex.sub(' ', x[1]).lower().split()))

In [10]:
keyAndListOfWords.take(1)

[('431949',
  ['black',
   'people',
   'and',
   'mormonismwhile',
   'at',
   'least',
   'two',
   'black',
   'men',
   'held',
   'the',
   'priesthood',
   'in',
   'the',
   'early',
   'church',
   'from',
   'the',
   'mid',
   's',
   'until',
   'the',
   'church',
   'of',
   'jesus',
   'christ',
   'of',
   'latter',
   'day',
   'saints',
   'lds',
   'church',
   'had',
   'a',
   'policy',
   'which',
   'prevented',
   'most',
   'men',
   'of',
   'black',
   'african',
   'descent',
   'from',
   'being',
   'ordained',
   'to',
   'the',
   'church',
   's',
   'lay',
   'priesthood',
   'under',
   'the',
   'temple',
   'and',
   'priesthood',
   'restrictions',
   'before',
   'black',
   'members',
   'of',
   'african',
   'descent',
   'could',
   'not',
   'receive',
   'the',
   'priesthood',
   'or',
   'participate',
   'in',
   'temple',
   'ordinances',
   'besides',
   'baptisms',
   'for',
   'the',
   'dead',
   'for',
   'a',
   'time',
   'in',
   

In [11]:
allWords = keyAndListOfWords.flatMap(lambda x: [(y, 1) for y in x[1]])

In [12]:
allWords.take(3)

[('black', 1), ('people', 1), ('and', 1)]

In [13]:
allCounts = allWords.reduceByKey(lambda x, y: x+y)

In [73]:
topWords = allCounts.top(10000, lambda x: x[1])

In [74]:
print("Top Words in Corpus:", allCounts.top(10, key=lambda x: x[1]))

Top Words in Corpus: [('the', 74530), ('of', 34512), ('and', 28479), ('in', 27758), ('to', 22583), ('a', 21212), ('was', 12160), ('as', 8811), ('for', 8773), ('on', 8435)]


In [75]:
topWordsK = sc.parallelize(range(10000))
dictionary = topWordsK.map (lambda x : (topWords[x][0], x))

In [76]:
print("Word Postions in our Feature Matrix. Last 20 words in 20k positions: ", dictionary.top(20, lambda x : x[1]))

Word Postions in our Feature Matrix. Last 20 words in 20k positions:  [('stealth', 9999), ('willie', 9998), ('replica', 9997), ('ricky', 9996), ('jackie', 9995), ('chooses', 9994), ('hosting', 9993), ('agreeing', 9992), ('endings', 9991), ('wednesday', 9990), ('personalities', 9989), ('stuff', 9988), ('exiles', 9987), ('relate', 9986), ('neighbor', 9985), ('spawned', 9984), ('ia', 9983), ('manpower', 9982), ('laying', 9981), ('clinic', 9980)]


In [77]:
allWordsWithDocID = keyAndListOfWords.flatMap(lambda x: ((j, x[0]) for j in x[1]))

In [78]:
allDictionaryWords = dictionary.join(allWordsWithDocID)

In [79]:
small_dict = dictionary.collectAsMap()

In [80]:
small_dict

{'the': 0,
 'of': 1,
 'and': 2,
 'in': 3,
 'to': 4,
 'a': 5,
 'was': 6,
 'as': 7,
 'for': 8,
 'on': 9,
 'is': 10,
 's': 11,
 'by': 12,
 'with': 13,
 'that': 14,
 'he': 15,
 'from': 16,
 'his': 17,
 'at': 18,
 'it': 19,
 'an': 20,
 'were': 21,
 'which': 22,
 'are': 23,
 'be': 24,
 'or': 25,
 'this': 26,
 'had': 27,
 'also': 28,
 'not': 29,
 'one': 30,
 'first': 31,
 'its': 32,
 'their': 33,
 'but': 34,
 'after': 35,
 'new': 36,
 'has': 37,
 'have': 38,
 'they': 39,
 'two': 40,
 'been': 41,
 'who': 42,
 'during': 43,
 'other': 44,
 'all': 45,
 'when': 46,
 'time': 47,
 'into': 48,
 'more': 49,
 'there': 50,
 'would': 51,
 'only': 52,
 'can': 53,
 'some': 54,
 'may': 55,
 'years': 56,
 'she': 57,
 'most': 58,
 'such': 59,
 'her': 60,
 'used': 61,
 'th': 62,
 'many': 63,
 'over': 64,
 'year': 65,
 'between': 66,
 'then': 67,
 'three': 68,
 'these': 69,
 'made': 70,
 'war': 71,
 'world': 72,
 'states': 73,
 'than': 74,
 'later': 75,
 'i': 76,
 'up': 77,
 'city': 78,
 'about': 79,
 'while': 

In [81]:
sc.broadcast(small_dict)

In [82]:
allWordsWithDocID.take(2)

[('black', '431949'), ('people', '431949')]

In [83]:
allDictionaryWords = allWordsWithDocID.map(lambda x: (x[0], (x[1], small_dict.get(x[0]))) if x[0] in\
                                           small_dict.keys() else None).filter(lambda x: x != None)

In [84]:
allDictionaryWords.take(3)

[('black', ('431949', 194)),
 ('people', ('431949', 103)),
 ('and', ('431949', 2))]

In [85]:
justDocAndPos = allDictionaryWords.map(lambda x: x[1])

In [86]:
justDocAndPos.take(3)

[('431949', 194), ('431949', 103), ('431949', 2)]

In [87]:
justDocAndPos.count()

889893

In [88]:
allDictionaryWordsInEachDoc = justDocAndPos.groupByKey()

In [89]:
allDictionaryWordsInEachDoc.take(2)

[('431949', <pyspark.resultiterable.ResultIterable at 0x7fa4bf1cfb50>),
 ('431952', <pyspark.resultiterable.ResultIterable at 0x7fa4bf1cf2d0>)]

In [90]:

def buildArray(listOfIndices):
  returnVal = np.zeros(10000)

  for index in listOfIndices:
    returnVal[index] = returnVal[index] + 1

  mysum = np.sum(returnVal)

  returnVal = np.divide(returnVal, mysum)
  return returnVal

In [91]:
allDocsAsNumpyArrays = allDictionaryWordsInEachDoc.map(lambda x: (x[0], buildArray(list(x[1]))))

In [92]:
allDocsAsNumpyArrays.count()

1000

In [93]:
print(allDocsAsNumpyArrays.take(3))

[('431949', array([0.07817024, 0.02999421, 0.02408801, ..., 0.        , 0.        ,
       0.        ])), ('431952', array([0.09178744, 0.03059581, 0.04830918, ..., 0.        , 0.        ,
       0.        ])), ('431959', array([0.09130435, 0.04347826, 0.03768116, ..., 0.        , 0.        ,
       0.        ]))]


In [94]:
zeroOne = allDocsAsNumpyArrays.map(lambda x: (x[0], np.array([0 if i == 0 else 1 for i in x[1]])))

In [96]:
dfArray = zeroOne.reduce(lambda x1, x2: ("", np.add(x1[1], x2[1])))[1]

In [97]:
dfArray

array([919, 920, 869, ...,   6,   5,   1])

In [99]:
multiplier = np.full(10000, numberOfDocs)

In [100]:
idfArray = np.log(np.divide(multiplier, dfArray))

In [101]:
allDocsAsNumpyArraysTFidf = allDocsAsNumpyArrays.map(lambda x: (x[0], np.multiply(x[1], idfArray)))

In [102]:
allDocsAsNumpyArraysTFidf.take(2)

[('431949',
  array([0.00660297, 0.00250097, 0.00338225, ..., 0.        , 0.        ,
         0.        ])),
 ('431952',
  array([0.00775321, 0.00255113, 0.0067832 , ..., 0.        , 0.        ,
         0.        ]))]

In [103]:
featuresRDD = wikiCats.join(allDocsAsNumpyArraysTFidf).map(lambda x: (x[1][0], x[1][1]))

In [104]:
wikiCats.join(allDocsAsNumpyArraysTFidf).take(1)

[('418623',
  ('Asteroid_spectral_classes',
   array([0.00721078, 0.00372845, 0.00399547, ..., 0.        , 0.        ,
          0.        ])))]

In [105]:
featuresRDD = wikiCats.join(allDocsAsNumpyArraysTFidf).map(lambda x: (x[1][0], x[1][1]))

featuresRDD.cache()
featuresRDD.take(10)

[('Asteroid_spectral_classes',
  array([0.00721078, 0.00372845, 0.00399547, ..., 0.        , 0.        ,
         0.        ])),
 ('S-type_asteroids',
  array([0.00721078, 0.00372845, 0.00399547, ..., 0.        , 0.        ,
         0.        ])),
 ('1918_ships',
  array([0.00535539, 0.00384469, 0.0052604 , ..., 0.        , 0.        ,
         0.        ])),
 ('All_Wikipedia_articles_written_in_Australian_English',
  array([0.00535539, 0.00384469, 0.0052604 , ..., 0.        , 0.        ,
         0.        ])),
 ('EngvarB_from_February_2018',
  array([0.00535539, 0.00384469, 0.0052604 , ..., 0.        , 0.        ,
         0.        ])),
 ('Maritime_incidents_in_1939',
  array([0.00535539, 0.00384469, 0.0052604 , ..., 0.        , 0.        ,
         0.        ])),
 ('Pages_using_deprecated_image_syntax',
  array([0.00535539, 0.00384469, 0.0052604 , ..., 0.        , 0.        ,
         0.        ])),
 ('S-class_destroyers_(1917)_of_the_Royal_Australian_Navy',
  array([0.00535539, 0

In [106]:
def getPrediction (textInput, k):
    # Create an RDD out of the textIput
    myDoc = sc.parallelize (('', textInput))

    # Flat map the text to (word, 1) pair for each word in the doc
    wordsInThatDoc = myDoc.flatMap (lambda x : ((j, 1) for j in regex.sub(' ', x).lower().split()))

    # This will give us a set of (word, (dictionaryPos, 1)) pairs
    allDictionaryWordsInThatDoc = dictionary.join (wordsInThatDoc).map (lambda x: (x[1][1], x[1][0])).groupByKey()

    # Get tf array for the input string
    myArray = buildArray (allDictionaryWordsInThatDoc.top (1)[0][1])

    #idf = np.log(np.divide(np.full(20000, numberOfDocs), myArray))

    # Get the tf * idf array for the input string
    myArray1 = np.multiply (myArray, idfArray)

    #myArray_total = list(myArray) + [0]*(20000-len(myArray))
    
    #allDocsAsNumpyArraysTFidf
    # Get the distance from the input text string to all database documents, using cosine similarity (np.dot() )
    distances = featuresRDD.map (lambda x : (x[0], np.dot (x[1], myArray1)))
    # distances = allDocsAsNumpyArraysTFidf.map (lambda x : (x[0], cousinSim (x[1],myArray)))
    # get the top k distances
    topK = distances.top (k, lambda x : x[1])
    
    # and transform the top k distances into a set of (docID, 1) pairs
    docIDRepresented = sc.parallelize(topK).map (lambda x : (x[0], 1))

    # now, for each docID, get the count of the number of times this document ID appeared in the top k
    numTimes = docIDRepresented.reduceByKey(lambda x, y: x+y)
    
    # Return the top 1 of them.
    # Ask yourself: Why we are using twice top() operation here?
    return numTimes.top(k, lambda x: x[1])

In [107]:
print(getPrediction('Sport Basketball Volleyball Soccer', 10))

[('Disambiguation_pages_with_short_description', 1), ('Human_name_disambiguation_pages', 1), ('Bullfighters', 1), ('Articles_containing_Japanese-language_text', 1), ('Japanese_martial_arts', 1), ('Ko-ryū_bujutsu', 1), ('All_article_disambiguation_pages', 1), ('All_disambiguation_pages', 1), ('Lists_of_sportspeople_by_sport', 1), ('Japanese_swordsmanship', 1)]


In [108]:
print(getPrediction('What is the capital city of Australia?', 10))

[('All_Wikipedia_articles_written_in_Australian_English', 2), ('All_set_index_articles', 2), ('Articles_with_short_description', 2), ('Use_Australian_English_from_April_2018', 1), ('Royal_Australian_Navy_ship_names', 1), ('Set_indices_on_ships', 1), ('Use_dmy_dates_from_April_2018', 1)]


Task-3 DataFrame

In [109]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate() 

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [110]:
catDF = wikiCats.toDF(["id", "Category"])

In [111]:
catDF.show()

+------+--------------------+
|    id|            Category|
+------+--------------------+
|434042|   1987_debut_albums|
|434042|Albums_produced_b...|
|434042|Articles_with_hAu...|
|434042|Articles_with_sho...|
|434042|CS1_German-langua...|
|434042|Cacophony_(band)_...|
|434042| Jason_Becker_albums|
|434042|Marty_Friedman_al...|
|434042|Shrapnel_Records_...|
|455070|All_articles_need...|
|455070|All_articles_with...|
|455070|Articles_needing_...|
|455070|Articles_with_OS_...|
|455070|Articles_with_dea...|
|455070|Articles_with_dea...|
|455070|Articles_with_per...|
|455070|Borough_of_Broxbo...|
|455070|            Cheshunt|
|455070|Commons_category_...|
|455070|Coordinates_on_Wi...|
+------+--------------------+
only showing top 20 rows



In [112]:
pages = keyAndText.toDF(["id", "Texts"])

In [113]:
pages.show()

+------+--------------------+
|    id|               Texts|
+------+--------------------+
|431949|Black people and ...|
|431952|Norwegian Militar...|
|431953|Doug Harvey (ice ...|
|431959|Staten Island Gre...|
|431962|USS Dale (DLG-19)...|
|431968|Cyclic guanosine ...|
|431969|Touch (disambigua...|
|431970|USS DaleUSS "Dale...|
|431971|Touch (manga)Touc...|
|431972|Touch (Unix)touch...|
|431975|Terry SawchukTerr...|
|431984|Island EdgeIsland...|
|431985|IBM i Control Lan...|
|431989|Echo Park, Los An...|
|431998|WattsA watt is a ...|
|431999|Lucien LaurinLuci...|
|432000|Arthur ComptonArt...|
|432001|Nose artNose art ...|
|432006|Bloomberg L.P.Blo...|
|432014|Franklin Delano R...|
+------+--------------------+
only showing top 20 rows



In [114]:
cat_pages = catDF.join(pages,catDF.id ==  pages.id,"inner")

In [115]:
from pyspark.sql import functions as func

In [116]:
category_count = cat_pages.groupby("Texts").agg(func.count("Category"))

In [117]:
category_count.show()

+--------------------+---------------+
|               Texts|count(Category)|
+--------------------+---------------+
|Brownian ratchetI...|             10|
|Messerschmitt Me ...|              8|
|Operator algebraI...|              6|
|USS S-26 (SS-131)...|             15|
|VeveyVevey () is ...|             34|
|II CorpsList of m...|              4|
|HillingHilling, e...|              9|
|Latin UnionThe La...|             30|
|Irwin CotlerIrwin...|             39|
|Rigid bodyIn phys...|              4|
|FaienceFaience or...|              7|
|Workers' Power (S...|              4|
|John Stafford (bi...|             20|
|Bob EhrlichRobert...|             29|
|Skull and crossbo...|             10|
|TrafficanteTraffi...|              5|
|Palace of the Duk...|              7|
|Zilveren Nipkowsc...|              5|
|James PlaskittJam...|             20|
|Laurette Séjourné...|             31|
+--------------------+---------------+
only showing top 20 rows



In [118]:
category_count.columns

['Texts', 'count(Category)']

In [119]:
category_count.agg({'count(Category)': 'mean'}).show()

+--------------------+
|avg(count(Category))|
+--------------------+
|  13.793793793793794|
+--------------------+



In [120]:
category_count.agg({'count(Category)': 'stddev'}).show()

+-----------------------+
|stddev(count(Category))|
+-----------------------+
|      11.06201871439371|
+-----------------------+



In [121]:
category_count.agg({'count(Category)': 'max'}).show()

+--------------------+
|max(count(Category))|
+--------------------+
|                  76|
+--------------------+



In [123]:
from pyspark.sql import DataFrameStatFunctions as statFunc
statFunc(category_count).approxQuantile('count(Category)', [0.5], 0.2)[0]

8.0

In [125]:
cat_pages.show()

+------+--------------------+------+--------------------+
|    id|            Category|    id|               Texts|
+------+--------------------+------+--------------------+
|434061|All_articles_lack...|434061|Leadership of Eas...|
|434061|Articles_containi...|434061|Leadership of Eas...|
|434061|Articles_lacking_...|434061|Leadership of Eas...|
|434061|East_Germany_poli...|434061|Leadership of Eas...|
|434061|Leaders_of_East_G...|434061|Leadership of Eas...|
|434061|Lists_of_politica...|434061|Leadership of Eas...|
|434061|Politics_of_East_...|434061|Leadership of Eas...|
|434061|Use_dmy_dates_fro...|434061|Leadership of Eas...|
|455037|All_article_disam...|455037|James LovellJames...|
|455037|All_disambiguatio...|455037|James LovellJames...|
|455037|Disambiguation_pa...|455037|James LovellJames...|
|455037|Human_name_disamb...|455037|James LovellJames...|
|455070|All_articles_need...|455070|CheshuntCheshunt ...|
|455070|All_articles_with...|455070|CheshuntCheshunt ...|
|455070|Articl

In [126]:
categories_num = cat_pages.groupby("Category").agg(func.count("Category"))

In [127]:
categories_num.show()

+--------------------+---------------+
|            Category|count(Category)|
+--------------------+---------------+
|Montreal_Expos_co...|              1|
|National_Baseball...|              4|
|Commodity_exchang...|              1|
|Wikipedia_article...|             34|
|         1965_deaths|              2|
|Use_British_Engli...|              1|
|Military_units_an...|              1|
|Journalists_from_...|              1|
|Cold_War_cruisers...|              1|
|CS1_Indonesian-la...|              1|
|Environmental_eco...|              1|
|Journalists_who_c...|              1|
|  Holocaust_theology|              1|
|EngvarB_from_Marc...|              1|
|Military_history_...|              1|
|   Decision_analysis|              1|
|              Heaven|              2|
|Articles_containi...|              1|
|British_Indian_Ar...|              1|
|Articles_with_fai...|              1|
+--------------------+---------------+
only showing top 20 rows



In [150]:
sorted_cat.select("Category").take(10)

[Row(Category='Articles_with_short_description'),
 Row(Category='All_articles_with_unsourced_statements'),
 Row(Category='Wikipedia_articles_with_WorldCat-VIAF_identifiers'),
 Row(Category='Wikipedia_articles_with_VIAF_identifiers'),
 Row(Category='Wikipedia_articles_with_LCCN_identifiers'),
 Row(Category='Commons_category_link_is_on_Wikidata'),
 Row(Category='Wikipedia_articles_with_ISNI_identifiers'),
 Row(Category='Wikipedia_articles_with_GND_identifiers'),
 Row(Category='All_articles_needing_additional_references'),
 Row(Category='Webarchive_template_wayback_links')]

In [138]:
categories_num.orderBy("count(Category)", descending=True).show()

+--------------------+---------------+
|            Category|count(Category)|
+--------------------+---------------+
| American_inventions|              1|
|Films_about_hosta...|              1|
|Birds_described_i...|              1|
|Canadian_late-nig...|              1|
|   Organic_gardening|              1|
|History_of_neuros...|              1|
|Wikipedia_article...|              1|
|Films_directed_by...|              1|
|Articles_containi...|              1|
|Use_Canadian_Engl...|              1|
|English_cricketer...|              1|
|         1870_births|              1|
|21st-century_Amer...|              1|
|Biology_journal_s...|              1|
|Webarchive_templa...|              1|
|Redirects_from_in...|              1|
|Roman_towns_and_c...|              1|
|Royal_Dutch_Shell...|              1|
|         2009_deaths|              1|
|American_female_c...|              1|
+--------------------+---------------+
only showing top 20 rows

